Copyright (c) 2018 Robert Bosch GmbH

All rights reserved.

This source code is licensed under the MIT license found in the
LICENSE file in the root directory of this source tree.

@author: Barbara Rakitsch

In [ ]:
%matplotlib notebook

import gpflow
import matplotlib.pylab as plt
import numpy as np
import utils
import sklearn.cluster

# Gaussian Process Regression

A Gaussian process is a collection of random variables, any finite number of which have a joint Gaussian distribution. It is completely specified by its mean function $m(x)$ and its covariance function $k(x,x′)$:

$f(x)∼GP(m(x),k(x,x′))$

In the following, we will show how to apply Gaussian Processes to a regression dataset using a zero mean function and a squared exponential kernel. The choice of the mean and the kernel function encodes our prior beliefs of the function.
In the training step, we restrict the fucntions of the prior such that they match the observations.

In [ ]:
N_train = 30 # number of training points
N_test = 100 # number of test points
num_samples = 1 # number of Monte Carlo samples in prediction
variance = 0.1 # variance of the noise

# generate synthetic features
X_test = np.linspace(0,10,N_test)[:,np.newaxis]
idx = np.random.randint(0, N_test, size=N_train)
X_train = X_test[idx]

# generate synthetic output by drawing a sample from the GP prior
kern = gpflow.kernels.SquaredExponential(1)
K = kern.compute_K_symm(X_train)
zeros = np.zeros(N_train)
f_train  = np.random.multivariate_normal(zeros, K, 1).T
# add observational noise
y_train  = f_train + np.sqrt(variance) * np.random.randn(N_train,1)

# initialize gaussian process regression object
gpr =  gpflow.models.gpr.GPR(X_train, y_train, kern)

# fit the parameters of the kernel and of the variance of the noise to the training data
opt = gpflow.train.ScipyOptimizer()
opt.minimize(gpr)

# predict to un-seen test data
Fhat_test = gpr.predict_f_samples(X_test, num_samples)[:,:,0].T # random samples of the GP posterior
Fhat_mean, Fhat_var = gpr.predict_f(X_test) # mean and variance of the predictions

# plot data
fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(111)
plt.plot(X_train, y_train, 'ko', markersize=3)
plt.plot(X_test, Fhat_test)
plt.fill_between(X_test[:,0], -2*np.sqrt(Fhat_var[:,0]) + Fhat_mean[:,0], +2*np.sqrt(Fhat_var[:,0]) + Fhat_mean[:,0], color='k', alpha=0.1)
utils.beautify_plot(ax, 'X', 'Y')




# Composite Kernels

The power of Gaussian Processes lies in choosing an expressive covariance function. In the following, we show how we can build powerful kernels by combining base kernels. 

In our example, we choose as covariance function a sum of a linear and a squared exponential kernel. The squared exponential kernel belongs to the class of stationary kernel functions - it takes only the relative distance between two data points into account. In case of extrapolation, the GP predictions fall back to the prior.
The linear kernel is equivalent to Bayesian linear regression and is a non-strationary kernel. It does not fall back to the prior when it comes to extrapolation.

In [ ]:
N_train = 100 # number of training points
N_test = 1000 # number of test points
num_samples = 1 # number of Monte Carlo samples in prediction
variance = 0.05 # variance of the noise

# generate synthetic features
X_test = np.linspace(0,10,N_test)[:,np.newaxis]
idx = np.random.randint(0, N_test/2, size=N_train)
X_train = X_test[idx]

# generate synthetic output by drawing samples from the base GP priors and adding them up
kern_lin = gpflow.kernels.Linear(1)
kern_se  = gpflow.kernels.SquaredExponential(1, variance=0.5, lengthscales=0.1)
K_lin = kern_lin.compute_K_symm(X_train)
K_se  = kern_se.compute_K_symm(X_train)
zeros = np.zeros(N_train)
f_lin = np.random.multivariate_normal(zeros, K_lin, 1).T
f_se  = np.random.multivariate_normal(zeros, K_se, 1).T
f_train = f_lin + f_se
# add observational noise
y_train  = f_train + np.sqrt(variance) * np.random.randn(N_train,1)

# initialize gaussian process regression object
kern = gpflow.kernels.Sum([kern_lin, kern_se]) # composite kernel
gpr =  gpflow.models.gpr.GPR(X_train, y_train, kern)

# fit the parameters of the kernel and of the variance of the noise to the training data
opt = gpflow.train.ScipyOptimizer()
opt.minimize(gpr)

# predict to un-seen test data
Fhat_test = gpr.predict_f_samples(X_test, num_samples)[:,:,0].T # random samples of the GP posterior
Fhat_mean, Fhat_var = gpr.predict_f(X_test) # mean and variance of the predictions

# plot data
fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(111)
plt.plot(X_train, y_train, 'ko', markersize=3)
plt.plot(X_test, Fhat_test)
plt.fill_between(X_test[:,0], -2*np.sqrt(Fhat_var[:,0]) + Fhat_mean[:,0], +2*np.sqrt(Fhat_var[:,0]) + Fhat_mean[:,0], color='k', alpha=0.1)
utils.beautify_plot(ax, 'X', 'Y')

# Sparse Gaussian Processes

Standard Gaussian Processes scale cubically with the number of data points $N$ which can be prohibitive for large-scale data sets. Sparse Gaussian Processes arrive at scalable solutions by capturing the information of the training data set in $M$ inducing points. The number of inducing points $M$ is a trade-off parameter between scalability and the quality of the approximation.

In [ ]:
# Noisy GP predictions
N_train = 100 # number of training points
N_test = 100 # number of test points
num_samples = 3 # number of Monte Carlo samples
variance = 0.1 # variance of the noise

# generate synthetic features
idx = np.random.randint(0, N_test, size=N_train)
X_test = np.linspace(0,10,N_test)[:,np.newaxis]
X_train = X_test[idx]

# generate synthetic output by drawing a sample from the GP prior 
kern = gpflow.kernels.SquaredExponential(1, lengthscales=0.4)
K = kern.compute_K_symm(X_train)
zeros = np.zeros(N_train)
f_train  = np.random.multivariate_normal(zeros, K, 1).T
# add observational noise
y_train  = f_train + np.sqrt(variance) * np.random.randn(N_train,1)

# standard Gaussian Process regression
gpr =  gpflow.models.gpr.GPR(X_train, y_train, kern)
gpr.likelihood.variance = variance
opt = gpflow.train.ScipyOptimizer()
opt.minimize(gpr)
Fhat_test = gpr.predict_f_samples(X_test, num_samples)[:,:,0].T
Fhat_mean, Fhat_var = gpr.predict_f(X_test)

fig = plt.figure(figsize=(6,6))
fig.subplots_adjust(bottom=0.2, left=0.2)
ax = fig.add_subplot(211)
plt.title('GPR')
plt.plot(X_train, y_train, 'ko', markersize=3)
plt.plot(X_test, Fhat_mean)
plt.fill_between(X_test[:,0], -2*np.sqrt(Fhat_var[:,0]) + Fhat_mean[:,0], +2*np.sqrt(Fhat_var[:,0]) + Fhat_mean[:,0], color='b', alpha=0.1)
utils.beautify_plot(ax)
plt.ylabel('Y')

# sparse Gaussian Process regression
M = 15
# inducing points are intialized by running K-Means over the training data
Z = sklearn.cluster.KMeans(M).fit(X_train).cluster_centers_ 
# initialize sparse Gaussian Process object
gpr = gpflow.models.sgpr.SGPR(X_train, y_train, kern, Z=Z)
# fit the parameters of the kernel and of the variance of the noise and the inducing points to the training data
opt = gpflow.train.ScipyOptimizer()
opt.minimize(gpr)
Fhat_test = gpr.predict_f_samples(X_test, num_samples)[:,:,0].T
Fhat_mean, Fhat_var = gpr.predict_f(X_test)

ax = fig.add_subplot(212)
plt.title('SGPR')
plt.plot(X_train, y_train, 'ko', markersize=3)
plt.plot(X_test, Fhat_mean)
plt.fill_between(X_test[:,0], -2*np.sqrt(Fhat_var[:,0]) + Fhat_mean[:,0], +2*np.sqrt(Fhat_var[:,0]) + Fhat_mean[:,0], color='b', alpha=0.1)
utils.beautify_plot(ax, 'X', 'Y')
for i in range(Z.shape[0]):
    plt.axvline(gpr.feature.Z.value[i], color='Orange', alpha=0.5)


# Non-linear Exogenous Gaussian Process regression

In the examples above, we used Gaussian Processes to analyse regression datasets. In this last example, we want to forecast
time-series data by taking the input history into account.

In [ ]:
nSteps = 1000 # number of time steps
T = np.linspace(0, 30, nSteps)[:,np.newaxis] # time points 
variance = 0.05 # variance of the noise

# generate synthetic features
kern = gpflow.kernels.SquaredExponential(1)
K = kern.compute_K_symm(T)
zeros = np.zeros(nSteps)
X = np.random.multivariate_normal(zeros, K, 1).T

# generate features by taking the history of the input into account
D = 5 # number of history events
delta = 10 # interval between history events
X_blowup = np.zeros((nSteps, D))
for i in range(D):
    if i==0:
        X_blowup[:,i] = X[:,0] # original inputs
    else:
        X_blowup[i*delta:,i] = X[:-(i*delta), 0] # inputs from past time point

# generate synthetic output by drawing a sample from the GP prior      
kern = gpflow.kernels.SquaredExponential(D)
K = kern.compute_K_symm(X_blowup)
zeros = np.zeros(nSteps)
F = np.random.multivariate_normal(zeros, K, 1).T
# add observational noise
Y = F + np.sqrt(variance) * np.random.randn(nSteps,1)

# splitting data ino training and test samples
idx_train = np.ones(nSteps, dtype=np.bool) 
idx_train[700:] = 0
T_train = T[idx_train]
T_test = T[~idx_train]
X_train = X[idx_train]
X_blowup_train = X_blowup[idx_train]
Y_train = Y[idx_train]
X_test  = X[~idx_train]
X_blowup_test = X_blowup[~idx_train]
Y_test  = Y[~idx_train]

# plot features
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(311)
plt.plot(T, X)
plt.xticks(())
utils.beautify_plot(ax, ylabel='X')

# model data not taking any history into account
kern = gpflow.kernels.SquaredExponential(1)
gpr =  gpflow.models.gpr.GPR(X_train, Y_train, kern)
opt = gpflow.train.ScipyOptimizer()
opt.minimize(gpr)
Fhat_mean, Fhat_var = gpr.predict_y(X)

ax = fig.add_subplot(312)
plt.title('GP')
plt.plot(T, F)
plt.plot(T_train, Y_train, '*', alpha=0.1)
utils.beautify_plot(ax, ylabel='Y')
plt.xticks(())
plt.fill_between(T[:,0], -2*np.sqrt(Fhat_var[:,0]) + Fhat_mean[:,0], +2*np.sqrt(Fhat_var[:,0]) + Fhat_mean[:,0], color='k', alpha=0.1)
utils.beautify_plot(ax, ylabel='Y')

# model data taking history into account
kern = gpflow.kernels.SquaredExponential(D)
gpr =  gpflow.models.gpr.GPR(X_blowup_train, Y_train, kern)
opt = gpflow.train.ScipyOptimizer()
opt.minimize(gpr)
Fhat_mean, Fhat_var = gpr.predict_y(X_blowup)

ax = fig.add_subplot(313)
plt.title('GP-NX')
plt.plot(T, F)
plt.plot(T_train, Y_train, '*', alpha=0.1)
plt.fill_between(T[:,0], -2*np.sqrt(Fhat_var[:,0]) + Fhat_mean[:,0], +2*np.sqrt(Fhat_var[:,0]) + Fhat_mean[:,0], color='k', alpha=0.1)
utils.beautify_plot(ax, 'T', 'Y')